# Eliminación de preguntas repetidas
Ya con todas las preguntas que cumplen los requisitos del INE para ser tomadas en cuenta, vamos 
a hacer otra depuración para eliminar aquellas preguntas que están repetidas.
Es importante considerar que dos preguntas pueden ser iguales aún cuando estén formuladas con 
palabras diferentes (es decir, una puede ser paráfrasis de la otra).
El resultado de este paso serán dos archivos:

- Uno que contenga el texto de las preguntas repetidas, así como una métrica que mida su parecido. De estas preguntas, sólo se conservará 1 en el dataset de trabajo.

  - La pregunta que será conservada es aquella que fue enviada primero de acuerdo al timestamp
          
- Un archivo con las preguntas depuradas (eliminando las repeticiones)

In [12]:
!pip install spacy
!pip install scikit-learn
!python -m spacy download es_core_news_md

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     --------------------------------------- 0.0/42.3 MB 217.9 kB/s eta 0:03:14
     --------------------------------------- 0.1/42.3 MB 327.7 kB/s eta 0:02:09
     --------------------------------------- 0.1/42.3 MB 595.3 kB/s eta 0:01:11
     ---------------------------------------- 0.3/42.3 MB 1.4 MB/s eta 0:00:31
      --------------------------------------- 0.6/42.3 MB 2.4 MB/s eta 0:00:18
     - -------------------------------------- 1.4/42.3 MB 4.4 MB/s eta 0:00:10
     -- ------------------------------------- 2.7/42.3 MB 7.4 MB/s eta 0:00:06
     --- ------------------------------------ 4.2/42.3 MB 10.6 MB/s eta 0:00:04
     ----- ---------------------------------- 6.2/42.3 MB 14.1 MB/s eta 0:00:03
     ------- -------------------------------- 8.3/42.3 MB 16.5 MB/s eta 0:00:03
     --------- ----------------------------- 10.3/42.3 MB 26

In [1]:
import pandas as pd
import spacy
from sklearn.metrics.pairwise import cosine_similarity

limpio_df = pd.read_csv('./Base_limpia.csv')

nlp = spacy.load('es_core_news_md')

preguntas = limpio_df['pregunta_norm']

vectores = [nlp(str(pregunta)).vector for pregunta in preguntas]

similitud = cosine_similarity(vectores)

repetidas = []

for index, fila in limpio_df.iterrows():
    for j in range(index + 1, len(preguntas)):
        if similitud[index, j] > 0.9:
            p1_fecha = limpio_df.loc[index]['fecha']
            p2_fecha = limpio_df.loc[j]['fecha']
            if p1_fecha < p2_fecha:
              repetidas.append(pd.DataFrame([[limpio_df.iloc[j]['pregunta'], limpio_df.loc[index]['pregunta_norm']]], index=None))
            else:
              repetidas.append(pd.DataFrame([[limpio_df.loc[index]['pregunta'], limpio_df.iloc[j]['pregunta_norm']]], index=None))
repetidas = pd.concat(repetidas)
repetidas.columns = ['repetida_de', 'pregunta']

display(repetidas)
repetidas.to_csv('./preguntas_repetidas.csv', index=False)

limpio_df = limpio_df[limpio_df['pregunta_norm'].isin(repetidas) == False]

,repetida_de,pregunta
0,En México muchas personas se quejan del sistem...,como podria usted cambiar sistema salud public...
0,En México muchas personas se quejan del sistem...,como podrias incrementar eficacia servicios sa...
0,En México muchas personas se quejan del sistem...,digan opinion respecto servicio salud mejor di...
0,En México muchas personas se quejan del sistem...,sistema salud insuficiente pregunta como podem...
0,En México muchas personas se quejan del sistem...,cual plan concreto mejorar acceso atencion med...
...,...,...
0,¿Cuál sería su enfoque para abordar la violenc...,candidato presidencial cuales propuestas comba...
0,¿Cómo abordará el problema de la desigualdad e...,como abordara brecha educativa zonas urbanas r...
0,¿Cuál es su enfoque para abordar la violencia ...,cual estrategia proponen prevenir violencia ge...
0,"¿Cuáles son sus propuestas para concientizar, ...",estrategias implementaran sensibilizar socieda...
